In [1]:
import numpy as np
import pandas as pd
from boruta import BorutaPy
from sklearn.datasets import load_diabetes
from sklearn.metrics import accuracy_score 
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [2]:
%cd S:\PPMI_RNA\Processed\Progression_Analysis\Male\HYS_1vs2vs3
data = pd.read_csv('CountData_HYS1vs2vs3_PD_withHYS_Male.csv')
data = data[data['HYS'] != 2]
data.head(1)

S:\PPMI_RNA\Processed\Progression_Analysis\Male\HYS_1vs2vs3


,HudAlphaID,HYS,ENSG00000223972.5,ENSG00000227232.5,ENSG00000278267.1,ENSG00000243485.5,ENSG00000284332.1,ENSG00000237613.2,ENSG00000268020.3,ENSG00000240361.2,...,ENSG00000273739.1,ENSG00000276700.1,ENSG00000276312.1,ENSG00000275757.1,ENSG00000278573.1,ENSG00000276017.1,ENSG00000278817.1,ENSG00000277196.4,ENSG00000278625.1,ENSG00000277374.1
3,5104_SL_0011,1,15,209,6,8,0,6,6,19,...,2,12,7,0,6,2,32,17,0,0


In [3]:
X = data.drop(['HudAlphaID', 'HYS'], axis = 1)
y = data['HYS']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .2, random_state = 1) 
print(X.head(2))

   ENSG00000223972.5  ENSG00000227232.5  ENSG00000278267.1  ENSG00000243485.5  \
3                 15                209                  6                  8   
9                  1                 55                  2                  0   

   ENSG00000284332.1  ENSG00000237613.2  ENSG00000268020.3  ENSG00000240361.2  \
3                  0                  6                  6                 19   
9                  0                  0                  0                  0   

   ENSG00000186092.6  ENSG00000238009.6  ...  ENSG00000273739.1  \
3                  7                176  ...                  2   
9                  0                 85  ...                  0   

   ENSG00000276700.1  ENSG00000276312.1  ENSG00000275757.1  ENSG00000278573.1  \
3                 12                  7                  0                  6   
9                  0                  3                  0                  0   

   ENSG00000276017.1  ENSG00000278817.1  ENSG00000277196.4  ENSG00

In [4]:
rf_all_features = RandomForestClassifier(random_state=1)
rf_all_features.fit(X_train, y_train)

RandomForestClassifier(random_state=1)

In [5]:
accuracy_score(y_test, rf_all_features.predict(X_test))

0.8775510204081632

In [6]:
rfc = RandomForestClassifier(random_state=1)
boruta_selector = BorutaPy(rfc, n_estimators= 'auto', verbose=1, random_state=1)
boruta_selector.fit(np.array(X_train), np.array(y_train))  

Iteration: 1 / 100
Iteration: 2 / 100
Iteration: 3 / 100
Iteration: 4 / 100
Iteration: 5 / 100
Iteration: 6 / 100
Iteration: 7 / 100
Iteration: 8 / 100
Iteration: 9 / 100
Iteration: 10 / 100
Iteration: 11 / 100
Iteration: 12 / 100
Iteration: 13 / 100
Iteration: 14 / 100
Iteration: 15 / 100
Iteration: 16 / 100
Iteration: 17 / 100
Iteration: 18 / 100
Iteration: 19 / 100
Iteration: 20 / 100
Iteration: 21 / 100
Iteration: 22 / 100
Iteration: 23 / 100
Iteration: 24 / 100
Iteration: 25 / 100
Iteration: 26 / 100
Iteration: 27 / 100
Iteration: 28 / 100
Iteration: 29 / 100
Iteration: 30 / 100
Iteration: 31 / 100
Iteration: 32 / 100
Iteration: 33 / 100
Iteration: 34 / 100
Iteration: 35 / 100
Iteration: 36 / 100
Iteration: 37 / 100
Iteration: 38 / 100
Iteration: 39 / 100
Iteration: 40 / 100
Iteration: 41 / 100
Iteration: 42 / 100
Iteration: 43 / 100
Iteration: 44 / 100
Iteration: 45 / 100
Iteration: 46 / 100
Iteration: 47 / 100
Iteration: 48 / 100
Iteration: 49 / 100
Iteration: 50 / 100
Iteration

BorutaPy(estimator=RandomForestClassifier(n_estimators=54,
                                          random_state=RandomState(MT19937) at 0x25E94CA3D40),
         n_estimators='auto',
         random_state=RandomState(MT19937) at 0x25E94CA3D40, verbose=1)

In [7]:
# print("Ranking: ",boruta_selector.ranking_)          
print("No. of significant features: ", boruta_selector.n_features_)

No. of significant features:  9


In [8]:
selected_rf_features = pd.DataFrame({'GeneID':list(X_train.columns),
                                       'Ranking':boruta_selector.ranking_})
selected_rf_features = selected_rf_features.sort_values(by='Ranking')

In [9]:
Top50 = selected_rf_features.iloc[:50, :]
Top50.head(3)

,CpG,Ranking
46672,ENSG00000108797.11,1
40128,ENSG00000237111.1,1
19480,ENSG00000135318.11,1


### Top 50

In [10]:
Top50.to_csv ('HYS1vs3_Boruta_Top50_Male.csv', index = False)
selected_rf_features.to_csv ('HYS1vs3_Boruta_All_Male.csv', index = False)

In [11]:
# create a list of column names to subset from data
col_names = Top50.iloc[:, 0].tolist()

# subset df based on the column names
df_subset = data.loc[:, data.columns.isin(col_names)]
df_subset['HYS'] = data['HYS']
df_subset.to_csv ('HYS1vs3_Boruta_Top50_Data_Male.csv', index = False)
df_subset.head(2)

C:\Users\system 4\AppData\Local\Temp\ipykernel_3356\1078700621.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset['HYS'] = data['HYS']


,ENSG00000116761.11,ENSG00000214711.9,ENSG00000270571.2,ENSG00000278962.1,ENSG00000155657.26,ENSG00000163492.15,ENSG00000116106.11,ENSG00000186235.10,ENSG00000154764.5,ENSG00000189283.9,...,ENSG00000240505.8,ENSG00000108797.11,ENSG00000183691.4,ENSG00000168646.12,ENSG00000104921.14,ENSG00000277301.1,ENSG00000263366.2,ENSG00000102349.17,ENSG00000215580.10,HYS
3,125,161,116,108,17276,1872,1284,63,276,839,...,216,267,306,426,123,97,195,362,772,1
9,67,68,45,56,7529,1217,938,21,125,447,...,117,85,202,186,57,82,126,195,554,1


### Top 100

In [12]:
Top100 = selected_rf_features.iloc[:100, :]
Top100.to_csv ('HYS1vs3_Boruta_Top100_Male.csv', index = False)
Top100.head(2)

,CpG,Ranking
46672,ENSG00000108797.11,1
40128,ENSG00000237111.1,1


In [13]:
# create a list of column names to subset from data
col_names = Top100.iloc[:, 0].tolist()

# subset df based on the column names
df_subset = data.loc[:, data.columns.isin(col_names)]
df_subset['HYS'] = data['HYS']
df_subset.to_csv ('HYS1vs3_Boruta_Top100_Data_Male.csv', index = False)
df_subset.head(2)

C:\Users\system 4\AppData\Local\Temp\ipykernel_3356\594748375.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset['HYS'] = data['HYS']


,ENSG00000117643.14,ENSG00000116983.12,ENSG00000157077.14,ENSG00000116641.17,ENSG00000116761.11,ENSG00000272824.1,ENSG00000117322.17,ENSG00000162817.6,ENSG00000214711.9,ENSG00000231918.1,...,ENSG00000104921.14,ENSG00000277301.1,ENSG00000184113.9,ENSG00000263366.2,ENSG00000075275.16,ENSG00000102349.17,ENSG00000158301.18,ENSG00000260585.1,ENSG00000215580.10,HYS
3,690,364,472,1411,125,10,328,95,161,15,...,123,97,209,195,465,362,142,9,772,1
9,233,216,363,1141,67,8,118,87,68,0,...,57,82,151,126,90,195,90,0,554,1


### Top 250

In [14]:
Top250 = selected_rf_features.iloc[:250, :]
Top250.to_csv ('HYS1vs3_Boruta_Top250_Male.csv', index = False)
Top250.head(2)

,CpG,Ranking
46672,ENSG00000108797.11,1
40128,ENSG00000237111.1,1


In [15]:
# create a list of column names to subset from data
col_names = Top250.iloc[:, 0].tolist()

# subset df based on the column names
df_subset = data.loc[:, data.columns.isin(col_names)]
df_subset['HYS'] = data['HYS']
df_subset.to_csv ('HYS1vs3_Boruta_Top250_Data_Male.csv', index = False)
df_subset.head(2)

C:\Users\system 4\AppData\Local\Temp\ipykernel_3356\969045065.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset['HYS'] = data['HYS']


,ENSG00000269737.2,ENSG00000228037.1,ENSG00000206652.1,ENSG00000117318.8,ENSG00000117643.14,ENSG00000116983.12,ENSG00000171793.14,ENSG00000157077.14,ENSG00000134709.10,ENSG00000116641.17,...,ENSG00000184867.13,ENSG00000158301.18,ENSG00000196507.10,ENSG00000133131.14,ENSG00000101842.13,ENSG00000260585.1,ENSG00000009694.13,ENSG00000235802.1,ENSG00000215580.10,HYS
3,10,28,27,190,690,364,1356,472,485,1411,...,154,142,108,411,443,9,1086,9,772,1
9,0,36,26,100,233,216,836,363,491,1141,...,61,90,85,265,339,0,1113,9,554,1


### Top 500

In [16]:
Top500 = selected_rf_features.iloc[:500, :]
Top500.to_csv ('HYS1vs3_Boruta_Top500_Male.csv', index = False)
Top500.head(2)

,CpG,Ranking
46672,ENSG00000108797.11,1
40128,ENSG00000237111.1,1


In [17]:
# create a list of column names to subset from data
col_names = Top500.iloc[:, 0].tolist()

# subset df based on the column names
df_subset = data.loc[:, data.columns.isin(col_names)]
df_subset['HYS'] = data['HYS']
df_subset.to_csv ('HYS1vs3_Boruta_Top500_Data_Male.csv', index = False)
df_subset.head(2)

C:\Users\system 4\AppData\Local\Temp\ipykernel_3356\1701761236.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset['HYS'] = data['HYS']


,ENSG00000205090.8,ENSG00000269737.2,ENSG00000228037.1,ENSG00000215788.10,ENSG00000171735.18,ENSG00000232208.2,ENSG00000116771.5,ENSG00000206652.1,ENSG00000169914.5,ENSG00000117318.8,...,ENSG00000196507.10,ENSG00000133131.14,ENSG00000101842.13,ENSG00000260585.1,ENSG00000009694.13,ENSG00000216171.1,ENSG00000235802.1,ENSG00000229745.1,ENSG00000215580.10,HYS
3,12,10,28,1949,814,44,188,27,1428,190,...,108,411,443,9,1086,0,9,1,772,1
9,7,0,36,1194,541,32,103,26,1094,100,...,85,265,339,0,1113,0,9,1,554,1
